In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings  #← OpenAIEmbeddings 가져오기
from langchain.vectorstores import Chroma  #← Chroma 가져오기
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chainlit as cl
import os
import tiktoken


# text_splitter = SpacyTextSplitter(
#     chunk_size=300, 
#     pipeline="ko_core_news_sm"
# )

def tiktoken_len(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    return len(tokens)
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=100,
    length_function=tiktoken_len
)

embeddings = OpenAIEmbeddings( #← OpenAIEmbeddings를 초기화
    model="text-embedding-ada-002" #← 모델명을 지정
)


database = Chroma(  #← Chroma를 초기화
    collection_name= "kei",
    persist_directory="./embedding",  #← 영속화 데이터 저장 위치 지정
    embedding_function=embeddings  #← 벡터화할 모델을 지정
)

doc_dir = "./data/"
file_list = os.listdir(doc_dir)

for f in file_list:
    loader = PyMuPDFLoader(doc_dir+f)
    documents = loader.load()
    splitted_documents = text_splitter.split_documents(documents)
    print(len(splitted_documents))
    database.add_documents(splitted_documents)

database.persist()
print("데이터베이스 생성이 완료되었습니다.") #← 완료 알림

/home/dyjin/work/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


2024-03-12 08:19:12 - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
2024-03-12 08:19:13 - loaded in 0 embeddings
2024-03-12 08:19:13 - loaded in 1 collections
284
2024-03-12 08:19:19 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-12 08:19:23 - Persisting DB to disk, putting it in the save folder: ./embedding
데이터베이스 생성이 완료되었습니다.
